In [1]:
import sys
sys.path.append("./expectation_value")

from expectationvalue import ExpVal
import numpy as np
import qiskit.quantum_info as qif
from qiskit_nature.second_q.mappers import JordanWignerMapper, QubitConverter

from qiskit_nature.second_q.transformers import FreezeCoreTransformer
from qiskit.algorithms.minimum_eigensolvers import NumPyMinimumEigensolver
from qiskit_nature.second_q.algorithms import GroundStateEigensolver
from qiskit_nature.units import DistanceUnit
from qiskit_nature.second_q.drivers import PySCFDriver

In [2]:
# Utils
def BinaryQubit( integer ):
    return list(bin(integer).replace("0b","")[::-1])

def DiffBit( bit1 , bit2 ):
    return  [ idx for idx , ( b1 , b2 ) in enumerate (zip( bit1,bit2 ) ) if b1!=b2   ]

def BinToState( binary ):
    rule = { 
        '0':np.array( [1,0] ),
        '1':np.array( [0,1] )
           }
    
    return reduce(np.kron,[rule[idx] for idx in binary])

In [3]:
class BasisSamplingExpectation():
    def __init__( self,
                 operator,
                 quantum_instance,
                ):
        """
        Estimates the expectation value of an observable OPERATOR on the state given by the circuit
        CIRCUIT_STATE. The samplings are performed on the QUANTUM_INSTANCE. The tolerance indicates
        the minimum proability allowed for a basis to be included on the R elements
        operator: OpFlow.OperatorBasis
        quantum_instance: QuantumInstace or Backend
        """
        BasisSamplingExpectation.quantum_instance = (
            quantum_instance if isinstance(quantum_instance,QuantumInstance) else
            QuantumInstance(backend = quantum_instance) 
        )
        BasisSamplingExpectation.operator = operator # add instances /opflow single operator
        
        
        
    def GetExpectation(self, circuit_state , R = 5):
        
        if len(circuit_state.clbits) == 0:
            circuit_state.add_register( ClassicalRegister( circuit_state.num_qubits ) )
        
        #dictionary with the counts with the best R  states
        weight_factors = self.WeightFactors( circuit_state )
        
        if R<len(weight_factors):
            weight_factors = dict( 
                sorted( weight_factors.items() , key = lambda x:x[1] , reverse = True)[:R] 
            )
            
        # returns the transicion matrix, is a matrix
        transition_matrix =  self.TransitionMatrix( weight_factors.keys() )
        
        #interference factors, a dictionary with the same keys of weight_factors
        # already the off diagonal terms are already divided by sqr(f_l)
        interference_factors = self.InterferenceFactors( weight_factors , circuit_state ) 
        
        #computes the new normalization
        norm_square = 1 / np.sum(
            list(weight_factors.values())
        )
        
        # Creates a matrix from the weights_factors, is the poduct of all its elements
        weight_matrix = np.array( [ [ 
            val_i*val_j for val_j in weight_factors.values()   
        ]  for val_i in weight_factors.values() ] )
        
        # Creates a matrix from the interference_factors, similar manner than from the weights 
        # except for the diagonal
        n = len(interference_factors)
        interference_matrix = np.zeros( (n,n) , dtype = complex )
        for (i,val_i) in enumerate(weight_factors):
            
            for (j,val_j) in enumerate(weight_factors):
                
                if i!=j:
                    interference_matrix[i,j] = (
                        interference_factors[val_j]*interference_factors[val_i].conjugate()
                    )
                    
                else:
                    interference_matrix[ i,j ] = weight_factors[val_i]
        exp_val = norm_square*weight_matrix * transition_matrix / interference_matrix
        
        print(exp_val.sum())
        
        return exp_val.sum()
        
        
    def WeightFactors(self, circuit_state):
        nq = circuit_state.num_qubits
        temp_circ = QuantumCircuit( nq , nq )
        
        temp_circ.compose( circuit_state , inplace=True )
        temp_circ.measure( temp_circ.qubits, temp_circ.clbits )
        
        trans_circ = self.quantum_instance.transpile( temp_circ )[0]
        results = self.quantum_instance.execute( trans_circ ).get_counts()
        #add non clbits case
        return { k:v/self.quantum_instance.run_config.shots for k,v in results.items() }
    
    def TransitionMatrix( self , basis ):
        states = [ of.StateFn(el) for el in basis ]
        return np.array([[ (i.adjoint()@self.operator@k).eval()   for i in states] for k in states] )
        
    def InterferenceFactors( self, weights , circuit_state ):
        
        states =list(weights.keys())
        state_l = states[0]
        interference_factors = { state_l : weights[state_l]  }
#         print(state_l)
        for state in  states[1:]:
#             print(state)
            A = self.InterferenceCircuitEvaluation( state_l , state , circuit_state , phase = True )
            B = self.InterferenceCircuitEvaluation( state_l , state , circuit_state , phase = False )
    
            interference_factors[state] =  A + B - (.5+.5j)*( 
                weights[state_l] + weights[state]
            )/np.sqrt( weights[state_l] )
        
        return interference_factors
    
    def InterferenceCircuitEvaluation( self , m , n , circuit_state , phase ):
        """
        m , n:: binary representation 
        phase :: boolean
        Return circuit  for estimating A and B
        """

        temp_circuit = QuantumCircuit( len(m) , len(m) )
        temp_circuit.compose(circuit_state , inplace=True)
        
        qbits = DiffBit( m , n )
        
        m = m[::-1]
        for qbit , value in enumerate(m):
            if value =='1':
                temp_circuit.x(qbit)

        for qbit in qbits[:-1]:
            temp_circuit.cnot(qbit,qbit+1)

        if phase:
            temp_circuit.s(qbits[0])

        temp_circuit.h(qbits[0])

        if phase:
            temp_circuit.x(qbits[0])
        
        temp_circuit.measure( temp_circuit.qubits , temp_circuit.clbits )
        trans_circ = self.quantum_instance.transpile( temp_circuit )
        
        results = self.quantum_instance.execute( trans_circ ).get_counts()
        results.setdefault( '0'*len(m), 0.)
        return results['0'*len(m)]/self.quantum_instance.run_config.shots 

In [4]:
class MoleculeHamiltonian():
    """
    Filler

    """
    
    def __init__(self,
                 molecule,
                 converter,
                 basis = 'sto3g',
                ):
        """

        Parameters
        ----------
        molecule : string.
            
        converter : QubitConverter.
            
        basis : string.
            

        """
        
        self.molecule = molecule
        self.basis = basis
        driver = PySCFDriver(
            atom= self.molecule,
            basis=self.basis,
            charge=0,
            spin=0,
            unit=DistanceUnit.ANGSTROM,
            )                                  # Define the PySCFDriver for a given molecule
        
        self.driver = driver
        self.problem = driver.run()            # Define the ElectronicStructureProblem from the driver
        self.converter = converter
        
    def Hamiltonian(self,
                    freeze_core = False,
                    remove_orbitals = None):
        
        
        """
        Filler


        Parameters
        ----------
        freeze_core : bool.
            
        remove_orbitals : List, optional.
            
            
        Returns
        -------
        qubit_op : PauliSumOp.
            
        """
        problem = self.problem
        fermionic_op = self.problem.hamiltonian.second_q_op()
            
        if freeze_core:
            fc_transformer = FreezeCoreTransformer(freeze_core=freeze_core, remove_orbitals=remove_orbitals)
            problem = fc_transformer.transform(problem)
            fermionic_op = problem.hamiltonian.second_q_op()
            
        qubit_op = self.converter.convert(fermionic_op,
                                           sector_locator=problem.symmetry_sector_locator)
        return qubit_op
    
    def ComputeGroundState(self , circuit = True):
        
        """
        Filler.


        Parameters
        ----------
        circuit : bool.
            


        Returns
        -------
        state : QuantumCircuit or array(2**num_qubits)
            
        eigenvalue : float.
            
        """
        solver = NumPyMinimumEigensolver()
        
        ground_solver = GroundStateEigensolver(self.converter, solver)
        ground_state = ground_solver.solve(self.problem)
        
        state = ground_state.groundstate[0]
        eigenvalue = ground_state.groundenergy
        
        if circuit:
            return state, eigenvalue
        else:
            return qif.Statevector(state).data, eigenvalue

In [5]:
def get_number_nbody_terms(hamiltonian):
    """
    Gets the needed data of the hamiltonian to compute the expectation values using ExpVal.exp_val().


    Parameters
    ----------
    hamiltonian : PauliSumOp.
        Hamiltonian.


    Returns
    -------
    bodies : list.
        N-bodies interactions.
    """
    
    n_bodies_inter = []
    
    for i in range(len(hamiltonian)):
        pauli_string = hamiltonian[i].to_pauli_op().primitive.to_label()
        num_x = pauli_string.count('X')
        num_y = pauli_string.count('Y')
        
        n_body_terms = num_x+num_y
        
        n_bodies_inter.append(n_body_terms)
    
    bodies = list(set(n_bodies_inter))
    
    return bodies


def get_obs_data(hamiltonian):
    """
    Gets the needed data of the hamiltonian to compute the expectation values using ExpVal.exp_val().


    Parameters
    ----------
    hamiltonian : PauliSumOp.
        Hamiltonian.


    Returns
    -------
    coeffs : array(num_paulis).
        Coefficients of each pauli string of the hamiltonian.
    obs : array(2, 2, n_qubits, num_paulis)
        Observable made of every pauli string of the hamiltonian.
    """
    
    num_paulis = len(hamiltonian)
    n_qubits = hamiltonian.num_qubits

    paulis_dict = {'X': np.array([[0., 1.], [1., 0.]], dtype="complex"),
               'Y': np.array([[0., -1.*1j], [1.*1j, 0.]], dtype="complex"),
               'Z': np.array([[1., 0], [0, -1.]], dtype="complex"),
               'I': np.array([[1., 0], [0, 1.]], dtype="complex")}

    obs = np.zeros((2, 2, n_qubits, num_paulis), dtype=complex)
    coeffs = np.zeros(num_paulis, dtype=complex)

    for i in range(num_paulis):
        pauli_string_coeff = hamiltonian[i].coeffs[0]
        pauli_string = hamiltonian[i].to_pauli_op().primitive.to_label()
        pauli_string_list = [paulis_dict[i] for i in pauli_string]
        obs[:,:,:,i] =  np.stack(pauli_string_list, axis=-1)
        coeffs[i] = pauli_string_coeff
    
    return coeffs, obs

In [6]:
dist = 1.0
molecules = {
    "H2Be": "Be .0 .0 .0; H .0 .0 -" + str(dist) + "; H .0 .0 " + str(dist),
    "H2":"H .0 .0 .0; H .0 .0 " + str(dist),
    "LiH":"Li .0 .0 .0; H .0 .0 " + str(dist),
    "H2O": "H -0.0399 -0.0038 0.0; O 1.5780 0.8540 0.0; H 2.7909 -0.5159 0.0",
    "NH4": "N 0.0 0.0 0.149; H 0.0 0.947 -0.348; H  0.821 -0.474 -0.348; H -0.821 -0.474 -0.348"
}

In [7]:
mol = 'H2'
# mol = 'LiH'
# mol = 'H2Be'
# mol = "H2O"
# mol = "NH4"

In [8]:
converter = QubitConverter( JordanWignerMapper(), z2symmetry_reduction=None)
molecular_hamiltonian = MoleculeHamiltonian(molecules[mol] , converter=converter, basis='sto3g')
hamiltonian = molecular_hamiltonian.Hamiltonian( freeze_core = False )

driver = molecular_hamiltonian.driver
problem = molecular_hamiltonian.problem
n_qubits = hamiltonian.num_qubits

bodies = get_number_nbody_terms(hamiltonian)

In [9]:
circ_gs, eig_gs = molecular_hamiltonian.ComputeGroundState()
eig_gs

-1.630327541152621

In [20]:
#########################################################################################################

In [11]:
class Custom_VQE():
    
    def __init__(self,
                 initial_point,
                 ansatz,
                 optimizer,
                 quantum_instance,
                ):
        self.initial_point = initial_point
        self.ansatz = ansatz
        self.optimizer = optimizer
        self.quantum_instance = quantum_instance
        
        
    def compute_minimum_eigenvalue( self, 
                                   operator, 
                                   exp_val,
                                   R=5,
                                  ):
        
        exp_val = exp_val( operator, self.quantum_instance )
        
        expectation_value = lambda x:exp_val.GetExpectation(
            self.ansatz.bind_parameters( x ),
            R=R,
        ).real
        
        results = optimizer.minimize( expectation_value , self.initial_point )
        return results
    
    def compute_minimum_eigenvalue_ours(self, operator):

        
        def exp_val_vqe(state, operator):
            
            coeffs, obs = get_obs_data(operator)
            algorithm = ExpVal(n_shots = 100000,
                    bodies = bodies, 
                    r = 4, 
                    r_shots = 1000,
                    n_qubits = n_qubits)
            
            algorithm.get_interferences(state)
            exp_val_paulis = algorithm.exp_val(obs)
            expectation_value = np.sum(exp_val_paulis*coeffs).real
            
            print(expectation_value)
            
            return expectation_value
        
        
        expectation_value = lambda x: exp_val_vqe(self.ansatz.bind_parameters(x), operator).real
        
        results = optimizer.minimize( expectation_value , self.initial_point )
        return results

In [12]:
from qiskit.circuit.library import EfficientSU2
from qiskit.algorithms.optimizers import SPSA, COBYLA

from qiskit_nature.second_q.circuit.library import UCC, UCCSD, HartreeFock



# ansatz = EfficientSU2(num_qubits=hamiltonian.num_qubits,
#                      reps = 1,
#                      entanglement = "linear",
#                      )

ansatz = UCCSD()
ansatz.num_particles = problem.num_particles
ansatz.num_spatial_orbitals = problem.num_spatial_orbitals
ansatz.qubit_converter = converter

initial_state = HartreeFock()
initial_state.num_particles = problem.num_particles
initial_state.num_spatial_orbitals = problem.num_spatial_orbitals
initial_state.qubit_converter = converter
ansatz.initial_state = initial_state

In [13]:
initial_point = np.random.randn( ansatz.num_parameters )

In [14]:
optimizer = COBYLA(maxiter=100)

In [15]:
from qiskit import Aer, QuantumCircuit, QuantumRegister, ClassicalRegister
from qiskit.utils import QuantumInstance
import qiskit.opflow as of
from functools import reduce

quantum_instance = QuantumInstance( Aer.get_backend("qasm_simulator") , shots=100000)

vqe_try = OurVQE(initial_point=initial_point,
                ansatz = ansatz,
                optimizer = optimizer,
                quantum_instance=quantum_instance)

In [16]:
eig_gs

-1.630327541152621

In [17]:
result_ours = vqe_try.compute_minimum_eigenvalue_ours( hamiltonian )

result_ours.fun

-1.074590001177982
-0.9963072196097014
-1.3216249013539159
-1.0087311704620887
-1.0493059203074617
-1.1603838291924042
-1.3392893068215568
-1.1337297911919357
-1.4321920186223902
-1.398424446501482
-1.516257977435892
-1.6061313053072834
-1.6193318649289068
-1.5511323790730076
-1.6270391484622184
-1.5367808516961845
-1.6300048187360014
-1.626431959016847
-1.6183532756034267
-1.6264746962992414
-1.6294475228827432
-1.6303912411386248
-1.631243686484861
-1.6317406917915966
-1.628868236297998
-1.6329808548621871
-1.6296965626157527
-1.6291010590775403
-1.6289440272492872
-1.6304505763968156
-1.6318684798774785
-1.6307343211701537
-1.6303976167518208
-1.631286752987932
-1.6286350430903989
-1.629415083816966
-1.6306406370509599
-1.6306273900339836
-1.6323970259434628
-1.6313472204361934
-1.6314340519944168
-1.6316190963497026
-1.6307509065236718
-1.6285080259173637


-1.6285080259173637

In [18]:
result = vqe_try.compute_minimum_eigenvalue( hamiltonian,
                                            exp_val= BasisSamplingExpectation, R=4)
result.fun

(-0.9772310492032168+0j)
(-1.0443337982682868+0j)
(-1.0717849059022382+0j)
(-0.9823556498774259+0j)
(-1.0933282171030083+0j)
(-1.1546246509899847+0j)
(-0.8258023783989255+0j)
(-0.7446554890569403+0j)
(-1.2786684618843593+0j)
(-1.402981856709562+0j)
(-1.4236459482951207+0j)
(-1.4005665709459638+0j)
(-1.3635896508774246+0j)
(-1.494144058951014+0j)
(-1.5690650941797135+0j)
(-1.545193218367299+0j)
(-1.5390546621964245+0j)
(-1.5071031149584617+0j)
(-1.5818200225417618+0j)
(-1.5337818895577424+0j)
(-1.607655994842777+0j)
(-1.600020953346244+0j)
(-1.5673177545773131+0j)
(-1.5878154372842563+0j)
(-1.5656400258419563+0j)
(-1.602408319652216+0j)
(-1.5731998886144345+0j)
(-1.5602507798779008+0j)
(-1.5696531709952721+0j)
(-1.5673651115876495+0j)
(-1.568476333716189+0j)
(-1.608144381613712+0j)
(-1.5720230443724326+0j)
(-1.5699884006491336+0j)
(-1.5717436823869644+0j)
(-1.570171073974853+0j)
(-1.5700966894069721+0j)
(-1.570768615546543+0j)
(-1.5701255310593145+0j)
(-1.5705662241034946+0j)
(-1.607627

-1.5709128584810224

In [19]:
from qiskit.algorithms import VQE

backend = Aer.get_backend("qasm_simulator")
quantum_instance = QuantumInstance(backend, shots=300000)


vqe = VQE(ansatz, optimizer=optimizer, quantum_instance=quantum_instance)

result = vqe.compute_minimum_eigenvalue(hamiltonian)

result.eigenvalue

(-1.6296522548386432+0j)